## No Trainer API
- 트레이너 api 없이 순수 파이토치로 작업해보기
- https://huggingface.co/learn/nlp-course/en/chapter3/4?fw=pt

In [5]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Found cached dataset glue (C:/Users/user/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at C:\Users\user\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-5137f2fb4737d116.arrow
Loading cached processed dataset at C:\Users\user\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-60ba0ce5ca5c6a38.arrow
Loading cached processed dataset at C:\Users\user\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-b26904918f30dd5d.arrow


### Prepare for training

- 데이터로더에 토큰들을 올려 배치를 나누기 전에 `tokenized_datasets`를 정리해야 한다.
- `Trainer`가 자동으로 해주던 작업을 스스로 해보자.
- 모델이 원하지 않는 칼럼들을 제거하자. sentence1, sentence2
- label를 labels로 이름을 변경하자. 모델은 labels라는 이름을 좋아한다.
- 리스트 대신 파이토치를 리턴할 수 있도록, 데이터셋의 포맷을 변경하자

In [22]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

In [24]:
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

- 데이터로더를 이용해서 데이터를 랜덤하게 섞어서 배치 사이즈에 맞게 나눈다.
- 콜래이터를 이용해서 파이토치 텐서로 변환하고, 배치 요소들 중 가장 긴 문장에 맞춰 패딩을 준다.

In [25]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)

- 과정이 잘 됐는지 배치 하나를 가져와서 확인해보자
- 배치의 사이즈는 각각 다 다르다. 데이터로더를 설정할 때 `shuffle=True`로 정해놓고 배치마다 맥시멈 길이가 다르게 패딩하기 때문이다.

In [35]:
len(train_dataloader)

459

In [26]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 82]),
 'token_type_ids': torch.Size([8, 82]),
 'attention_mask': torch.Size([8, 82])}

In [27]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

- 모든 트랜스포머 모델은 레이블이 주어지면 로스를 리턴한다.
- 배치의 각 인풋당 2개의 로짓이 할당된다.

In [28]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

tensor(0.9381, grad_fn=<NllLossBackward0>) torch.Size([8, 2])


- `AdamW` 옵티마이저는 Adam과 비슷한데, 웨이트 감쇠 정규화를 위한 트위스트가 포함된다. 자세한 문서: https://arxiv.org/abs/1711.05101

In [29]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5) # 0.00005

c:\Users\user\miniconda3\envs\gpu2\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


- `linear decay`는 최대 5e-5에서 0까지 러닝레이트 스케줄러가 이용할 것이다.
- 제대로 정의하기 위해서는 정확한 `training_step`을 알아야 한다.
- `training_step` = 에포크 * 전체 배치들의 수. 
- `train_dataloader`의 크기는 전체 배치들의 수다.

In [30]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

1377


### The training loop

- GPU 이용하자. 몇 시간 걸릴 작업 몇 분으로 줄여준다.

In [34]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

- 학습을 할 수 있는 준비가 끝났다.
- `tqdm` 라이브러리는 진행된 정도를 표현해준다.
<br><br>
- 에포크 한 번당 배치 459개가 각각 학습
- 배치 하나를 꺼내서 딕셔너리를 만들어주는데, 밸류는 GPU에게 작업을 맡긴다.
- 배치 딕셔너리를 모델에 넣고 학습을 시킨다.
- 손실율을 확인하고 역전파 시행
- 옵티마이저 적용하고
- 러닝레이트 스케줄러로 러닝레이트 적용
- 옵티마이저 초기화하고
- 프로그래스바로 표현
<br><br>
- 

In [36]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/1377 [00:00<?, ?it/s]

KeyboardInterrupt: 

#### The evaluation loop

- `Evalute` 라이브러리 통해서 메트릭을 이용하자.
- `add_batch` 메서드는 루프를 돌면서 배치를 축적한다.
- 결과는 헤드 초기화와 데이터 셔플링 때문에 경우마다 다르다.

In [37]:
import evaluate

metric = evaluate.load("glue", "mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.8284313725490197, 'f1': 0.8809523809523808}

### Accerlate

- `accerlate` 라이브러리를 이용하면 몇가지 설정을 통해 학습을 여러 GPU나 TPU에 분산시킬 수 있다. 
- 아래 코드는 기본 구조

In [ ]:
from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
optimizer = AdamW(model.parameters(), lr=3e-5)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

- 엑셀러레이트를 이용한 코드

- 관련링크: https://huggingface.co/learn/nlp-course/en/chapter3/4?fw=pt
- 나중에 다시 확인하러 오겠음
- ```
    The first line to add is the import line. The second line instantiates an Accelerator object that will look at the environment and initialize the proper distributed setup. 🤗 Accelerate handles the device placement for you, so you can remove the lines that put the model on the device (or, if you prefer, change them to use accelerator.device instead of device).

    Then the main bulk of the work is done in the line that sends the dataloaders, the model, and the optimizer to accelerator.prepare(). This will wrap those objects in the proper container to make sure your distributed training works as intended. The remaining changes to make are removing the line that puts the batch on the device (again, if you want to keep this you can just change it to use accelerator.device) and replacing loss.backward() with accelerator.backward(loss).
    ```

In [ ]:
from accelerate import Accelerator
from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler

accelerator = Accelerator() # 1

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
optimizer = AdamW(model.parameters(), lr=3e-5)

# 2
train_dl, eval_dl, model, optimizer = accelerator.prepare( 
    train_dataloader, eval_dataloader, model, optimizer
)

num_epochs = 3
num_training_steps = num_epochs * len(train_dl)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dl:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss) # 3

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

- vscode말고 다른 환경에서 사용할 수도 있다.
- 터미널: `accelerate config`, `accelerate launch train.py`
- 코랩에서 사용시:
- ```python
    from accelerate import notebook_launcher

    notebook_launcher(training_function)
    ```
---